In [ ]:
import tensorflow as tf 
import tensorflow_datasets as tfds
import numpy as np 
import matplotlib.pyplot as plt

2026-01-21 17:34:24.953380: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
dataset=tfds.load('imdb_reviews',as_supervised=True)
train_dataset,test_dataset=dataset['train'],dataset['test']

batch_size=32
train_dataset=train_dataset.shuffle(1000).batch(batch_size)
test_dataset=test_dataset.batch(batch_size)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

In [ ]:
example,label=next(iter(train_dataset))
print(example.numpy()[0])
print(label.numpy()[0])

In [ ]:
encoder=tf.keras.layers.TextVectorization(max_tokens=10000)
encoder.adapt(train_dataset.map(lambda text,labels:text))
vocabulary=np.array(encoder.get_vocabulary())


orginal_text=example.numpy()[0]
encoded_text=encoder(orginal_text).numpy()
decoded_text=' '.join(vocabulary[encoded_text])
print(orginal_text)
print(decoded_text) 


In [ ]:
moddel=tf.keras.Sequential([
    tf.keras.layers.Embedding(input_shape=(1,),dtype=tf.),
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()),64,mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(1)
    
    
    
    
    
    ])



In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])


In [ ]:
history=model.fit(train_dataset,epochs=5,validation_data=test_dataset)

In [ ]:
from matplotlib import pyplot as plt

history_dict=history.history
acc=history_dict['accuracy']
val_acc=history_dict['val_accuracy']
val_loss=history_dict['val_loss']
loss=history_dict['loss']

plt.figure(figsize=(12,9))
plt.subplot(1,2,1)
plt.plot(acc,label='Training Accuracy')
plt.plot(val_acc,label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(loss,label='Training Loss')
plt.plot(val_loss,label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

testing the trained model

In [ ]:
sample_text=["This movie was awesome! The acting was great, plot was wonderful, and there were pythons...so yea!"]
predicted=model.predict(sample_text)
print(predicted)